In [ ]:
import pandas as pd
import numpy as np
import os
import ray
import time
import glob
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional


@ray.remote
class ProgressTracker:
    """진행률 추적을 위한 Ray Actor"""
    def __init__(self, total_files: int):
        self.total_files = total_files
        self.completed_files = 0
        self.failed_files = 0
        self.start_time = time.time()
        
    def update_progress(self, success: bool = True):
        if success:
            self.completed_files += 1
        else:
            self.failed_files += 1
            
        progress = (self.completed_files + self.failed_files) / self.total_files * 100
        elapsed = time.time() - self.start_time
        
        if (self.completed_files + self.failed_files) > 0:
            avg_time = elapsed / (self.completed_files + self.failed_files)
            remaining = (self.total_files - self.completed_files - self.failed_files) * avg_time
            
            print(f"📊 진행률: {progress:.1f}% | 성공: {self.completed_files} | 실패: {self.failed_files} | "
                  f"예상 남은 시간: {format_duration(remaining)}")
    
    def get_stats(self):
        return {
            'total_files': self.total_files,
            'completed': self.completed_files,
            'failed': self.failed_files,
            'total_duration': time.time() - self.start_time
        }


@ray.remote
def preprocess_batch_parallel(batch_data: pd.DataFrame, car_info: Dict[str, Any]) -> pd.DataFrame:
    """배치 데이터 병렬 전처리"""
    df = batch_data.copy()
    
    # 필드명에서 공백 제거
    df.columns = df.columns.str.replace(' ', '')
    cols = list(df.columns)
    
    # 필드 순서를 알파벳 순으로 재정렬 (clientid, timestamp는 앞에 유지)
    cols_sorted = cols[0:2] + sorted(cols[2:])
    df = df[cols_sorted]
    
    # ' None' 또는 'None'을 NaN으로 변경
    df.replace([' None', 'None'], np.nan, inplace=True)
    
    # 오류 값 처리 (clientid, timestamp 제외)
    target_cols = cols_sorted[2:]
    
    # 숫자 값을 float 타입으로 변경
    for field in target_cols:
        try:
            df[field] = df[field].astype(float)
        except:
            continue
    
    # SOC, SOH 필드 검증 (0~100 범위)
    soc_soh_cols = [col for col in target_cols if 'soc' in col or 'soh' in col]
    for col in soc_soh_cols:
        df[col] = df[col].map(lambda val: np.nan if (isinstance(val, (int, float)) and (val < 0 or val > 100)) else val)
    
    # 전압 필드 검증 (3000 이하)
    voltage_cols = [col for col in target_cols if '_v' in col]
    for col in voltage_cols:
        df[col] = df[col].map(lambda val: np.nan if (isinstance(val, (int, float)) and val > 3000) else val)
    
    # 온도 필드 검증 (-35 ~ 80도)
    temp_cols = [col for col in target_cols if 'temperature' in col]
    for col in temp_cols:
        df[col] = df[col].map(lambda val: np.nan if (isinstance(val, (int, float)) and (val < -35 or val > 80)) else val)
    
    # 전류 필드 검증 (-500 ~ 500)
    current_cols = [col for col in target_cols if 'curr' in col or col.startswith('cell')]
    for col in current_cols:
        df[col] = df[col].map(lambda val: np.nan if (isinstance(val, (int, float)) and (val < -500 or val > 500)) else val)
    
    # 속도 필드 검증 (0 ~ 180)
    speed_cols = [col for col in target_cols if 'speed' in col]
    for col in speed_cols:
        df[col] = df[col].map(lambda val: np.nan if (isinstance(val, (int, float)) and (val < 0 or val > 180)) else val)
    
    # 누적주행거리 필드 검증 (0 초과 ~ 2,000,000 이하)
    mileage_cols = [col for col in target_cols if 'mileage' in col]
    for col in mileage_cols:
        df[col] = df[col].map(lambda val: np.nan if (isinstance(val, (int, float)) and (val <= 0 or val > 2000000)) else val)
    
    # 차종 정보 추가
    df['car_type'] = car_info.get('car_type', np.nan)
    df['model_year'] = car_info.get('model_year', np.nan)
    df['model_month'] = car_info.get('model_month', np.nan)
    
    return df


@ray.remote
def preproc_betterwhy_parallel(csv_file: str, output_dir: str, remove_duplicates: bool = True, 
                              progress_tracker=None) -> Dict[str, Any]:
    """
    단일 CSV 파일을 병렬로 전처리
    """
    try:
        print(f"🔄 처리 시작: {os.path.basename(csv_file)}")
        file_start_time = time.time()
        
        # 출력 디렉토리 생성
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        # 배치 처리 설정
        batch_size = 30000  # 병렬처리시 메모리 효율을 위해 조금 줄임
        skip_rows = 0
        batch_futures = []
        car_info = {}
        
        # 첫 번째 배치에서 차량 정보 가져오기
        first_batch = pd.read_csv(csv_file, nrows=1000, low_memory=False)
        if not first_batch.empty:
            carid = first_batch.iloc[0]["clientid"]
            
            # 차량 타입 정보 로드
            try:
                carid_df = pd.read_csv("/Users/moon/ev_streamlit/preproc/betterwhy_cartype_list.csv")
                result = carid_df[carid_df['client_id'] == carid]
                
                if len(result) > 0:
                    row = result.iloc[0]
                    car_info = {
                        'car_type': row['car_type'],
                        'model_year': row['model_year'],
                        'model_month': row['model_month']
                    }
                else:
                    print(f"{os.path.basename(csv_file)}: clientid로 car_type을 찾을 수 없습니다.")
                    car_info = {'car_type': np.nan, 'model_year': np.nan, 'model_month': np.nan}
            except Exception as e:
                print(f"차량 타입 정보 로드 실패: {e}")
                car_info = {'car_type': np.nan, 'model_year': np.nan, 'model_month': np.nan}
        
        # CSV 파일을 배치로 읽어서 병렬 처리
        while True:
            try:
                df = pd.read_csv(csv_file, skiprows=skip_rows, nrows=batch_size, low_memory=False)
                
                if df.empty:
                    break
                
                # 배치를 병렬로 전처리
                batch_future = preprocess_batch_parallel.remote(df, car_info)
                batch_futures.append(batch_future)
                
                skip_rows += batch_size
                
            except Exception as e:
                if 'No columns to parse from file' in str(e):
                    break
                else:
                    raise e
        
        if not batch_futures:
            if progress_tracker:
                progress_tracker.update_progress.remote(False)
            return {'success': False, 'error': '처리할 데이터가 없습니다.'}
        
        # 모든 배치 처리 결과 수집
        print(f"📦 {os.path.basename(csv_file)}: {len(batch_futures)}개 배치 병렬 처리 완료, 결합 중...")
        processed_batches = ray.get(batch_futures)
        
        # 모든 배치 데이터 결합
        final_df = pd.concat(processed_batches, ignore_index=True)
        
        # 시간순 정렬
        final_df.sort_values(by='timestamp', inplace=True)
        final_df.reset_index(drop=True, inplace=True)
        
        # Client ID별로 그룹화하여 저장
        saved_files = save_by_client_id_parallel(final_df, output_dir, remove_duplicates)
        
        processing_time = time.time() - file_start_time
        
        if saved_files:
            # 저장 성공 시 원본 파일 삭제
            os.remove(csv_file)
            
            # 원본 파일이 있던 디렉토리가 비어있으면 삭제
            original_dir = os.path.dirname(csv_file)
            if original_dir and not os.listdir(original_dir):
                os.rmdir(original_dir)
            
            result = {
                'success': True,
                'file': csv_file,
                'saved_files': len(saved_files),
                'rows_processed': len(final_df),
                'processing_time': processing_time
            }
            
            print(f"✅ {os.path.basename(csv_file)}: 완료 ({len(final_df):,}행, {format_duration(processing_time)})")
            
            if progress_tracker:
                progress_tracker.update_progress.remote(True)
            
            return result
        else:
            if progress_tracker:
                progress_tracker.update_progress.remote(False)
            return {'success': False, 'error': '파일 저장 실패'}
            
    except Exception as e:
        if progress_tracker:
            progress_tracker.update_progress.remote(False)
        return {'success': False, 'error': str(e), 'file': csv_file}


def save_by_client_id_parallel(df: pd.DataFrame, output_dir: str, remove_duplicates: bool = True) -> List[str]:
    """
    Client ID별로 CSV 파일 저장 (병렬처리 버전)
    """
    saved_files = []
    
    try:
        # Client ID별로 그룹화
        grouped = df.groupby('clientid')
        
        for client_id, new_data in grouped:
            # 시작 날짜 추출
            try:
                new_data_copy = new_data.copy()
                new_data_copy['timestamp'] = pd.to_datetime(new_data_copy['timestamp'])
                start_date = new_data_copy['timestamp'].min()
                
                year_month = start_date.strftime('%y%m')
                start_date_str = start_date.strftime('%y%m%d')
                
            except Exception as e:
                print(f"날짜 변환 오류 (Client {client_id}): {e}")
                now = datetime.now()
                year_month = now.strftime('%y%m')
                start_date_str = now.strftime('%y%m%d')
            
            # 월별 폴더 경로 생성
            monthly_dir = os.path.join(output_dir, year_month)
            os.makedirs(monthly_dir, exist_ok=True)
            
            # 파일명 생성
            safe_client_id = str(client_id).replace('/', '_').replace('\\', '_')
            filename = f"{safe_client_id}_{start_date_str}.csv"
            filepath = os.path.join(monthly_dir, filename)
            
            # 기존 파일들 확인
            existing_files = find_existing_client_files(output_dir, safe_client_id)
            
            if existing_files:
                # 기존 파일들과 병합
                combined_data = merge_with_existing_files(existing_files, new_data, client_id, remove_duplicates)
                save_data_by_month(combined_data, output_dir, safe_client_id)
            else:
                # 새 파일 생성
                if remove_duplicates:
                    new_data = remove_duplicate_records(new_data, client_id)
                
                new_data.to_csv(filepath, index=False, encoding='utf-8')
            
            saved_files.append(filepath)
        
        return saved_files
        
    except Exception as e:
        print(f"파일 저장 중 오류: {e}")
        return []


def find_existing_client_files(output_dir: str, safe_client_id: str) -> List[str]:
    """특정 클라이언트의 기존 파일들을 모두 찾기"""
    existing_files = []
    pattern = os.path.join(output_dir, "*", f"clientid_{safe_client_id}_*.csv")
    existing_files = glob.glob(pattern)
    return existing_files


def merge_with_existing_files(existing_files: List[str], new_data: pd.DataFrame, 
                            client_id: str, remove_duplicates: bool) -> pd.DataFrame:
    """기존 파일들과 새 데이터를 병합"""
    try:
        all_data = [new_data]
        total_existing_rows = 0
        
        for file_path in existing_files:
            try:
                existing_df = pd.read_csv(file_path, low_memory=False)
                all_data.append(existing_df)
                total_existing_rows += len(existing_df)
            except Exception as e:
                print(f"파일 로드 실패 {file_path}: {e}")
        
        # 모든 데이터 결합
        combined_data = pd.concat(all_data, ignore_index=True)
        
        # 중복 제거
        if remove_duplicates:
            combined_data = remove_duplicate_records(combined_data, client_id)
        
        # 시간순 정렬
        combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'])
        combined_data.sort_values(by='timestamp', inplace=True)
        combined_data.reset_index(drop=True, inplace=True)
        
        # 기존 파일들 삭제
        for file_path in existing_files:
            try:
                os.remove(file_path)
            except Exception as e:
                print(f"파일 삭제 실패 {file_path}: {e}")
        
        return combined_data
        
    except Exception as e:
        print(f"파일 병합 중 오류 (Client {client_id}): {e}")
        return new_data


def save_data_by_month(df: pd.DataFrame, output_dir: str, safe_client_id: str):
    """데이터를 월별로 분할하여 CSV로 저장"""
    try:
        # 월별로 그룹화
        df['year_month'] = df['timestamp'].dt.strftime('%y%m')
        monthly_groups = df.groupby('year_month')
        
        for year_month, month_data in monthly_groups:
            start_date_str = month_data['timestamp'].min().strftime('%y%m%d')
            
            monthly_dir = os.path.join(output_dir, year_month)
            os.makedirs(monthly_dir, exist_ok=True)
            
            filename = f"{safe_client_id}_{start_date_str}.csv"
            filepath = os.path.join(monthly_dir, filename)
            
            month_data_clean = month_data.drop('year_month', axis=1)
            month_data_clean.to_csv(filepath, index=False, encoding='utf-8')
        
    except Exception as e:
        print(f"월별 저장 중 오류: {e}")


def remove_duplicate_records(df: pd.DataFrame, client_id: str) -> pd.DataFrame:
    """중복 레코드 제거 (timestamp 기준)"""
    try:
        df_deduplicated = df.drop_duplicates(subset=['clientid', 'timestamp'], keep='last')
        
        original_count = len(df)
        final_count = len(df_deduplicated)
        
        if original_count != final_count:
            print(f"  Client {client_id}: 중복 제거 ({original_count} -> {final_count})")
        
        return df_deduplicated
        
    except Exception as e:
        print(f"  중복 제거 중 오류 (Client {client_id}): {e}")
        return df


def process_multiple_files_parallel(csv_files: List[str], output_dir: str = "processed", 
                                   remove_duplicates: bool = True, max_workers: Optional[int] = None) -> Dict[str, Any]:
    """
    Ray를 사용한 병렬 파일 처리
    
    Args:
        csv_files: 처리할 CSV 파일 리스트
        output_dir: 출력 디렉토리
        remove_duplicates: 중복 데이터 제거 여부
        max_workers: 최대 워커 수 (None이면 CPU 코어 수 사용)
    """
    
    # Ray 초기화
    if not ray.is_initialized():
        # CPU 코어 수의 80% 사용 (시스템 안정성을 위해)
        num_cpus = max_workers or max(1, int(os.cpu_count() * 0.8))
        ray.init(num_cpus=num_cpus, ignore_reinit_error=True)
        print(f"🚀 Ray 초기화 완료 (워커 수: {num_cpus})")
    
    total_files = len(csv_files)
    print(f"📁 총 {total_files}개 파일 병렬 처리 시작")
    print(f"📂 출력 디렉토리: {output_dir}")
    print("=" * 80)
    
    # 진행률 추적기 생성
    progress_tracker = ProgressTracker.remote(total_files)
    
    # 전체 처리 시작 시간
    total_start_time = time.time()
    
    # 모든 파일을 병렬로 처리
    file_futures = []
    for csv_file in csv_files:
        future = preproc_betterwhy_parallel.remote(
            csv_file, output_dir, remove_duplicates, progress_tracker
        )
        file_futures.append(future)
    
    # 모든 작업 완료 대기
    print("⏳ 모든 파일 병렬 처리 중...")
    results = ray.get(file_futures)
    
    # 최종 통계
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    final_stats = ray.get(progress_tracker.get_stats.remote())
    
    successful_files = sum(1 for r in results if r.get('success', False))
    failed_files = len(results) - successful_files
    total_rows = sum(r.get('rows_processed', 0) for r in results if r.get('success', False))
    
    print("\n" + "=" * 80)
    print("🎉 병렬 처리 완료!")
    print(f"📈 성공: {successful_files}개 | ❌ 실패: {failed_files}개")
    print(f"📊 총 처리 행 수: {total_rows:,}행")
    print(f"⏱️  총 소요시간: {format_duration(total_duration)}")
    print(f"⚡ 평균 파일당: {format_duration(total_duration / total_files)}")
    
    # 실패한 파일들 출력
    if failed_files > 0:
        print(f"\n❌ 실패한 파일들:")
        for result in results:
            if not result.get('success', False):
                file_name = os.path.basename(result.get('file', 'Unknown'))
                error = result.get('error', 'Unknown error')
                print(f"   - {file_name}: {error}")
    
    return {
        'total_files': total_files,
        'successful': successful_files,
        'failed': failed_files,
        'total_duration': total_duration,
        'avg_duration_per_file': total_duration / total_files,
        'total_rows_processed': total_rows,
        'results': results
    }


def format_duration(seconds: float) -> str:
    """초를 읽기 쉬운 형태로 변환"""
    if seconds < 60:
        return f"{seconds:.1f}초"
    elif seconds < 3600:
        minutes = int(seconds // 60)
        secs = seconds % 60
        return f"{minutes}분 {secs:.1f}초"
    else:
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        return f"{hours}시간 {minutes}분 {secs:.1f}초"



In [3]:

# 사용 예시
if __name__ == "__main__":
    # CSV 파일 리스트 (실제 파일 경로로 수정하세요)
    # csv_files = [
    #     # "/path/to/your/file1.csv",
    #     # "/path/to/your/file2.csv",
    #     # 또는 glob 패턴 사용:
    #     # *glob.glob("/path/to/your/data/*.csv")
    # ]
    
    csv_files = csv_files = glob.glob("/Volumes/Data/betterwhy_origin/20250716_0722(300대)/**/*.csv", recursive=True)
    # 병렬 처리 실행 (CPU 코어 수의 80% 사용)
    results = process_multiple_files_parallel(
        csv_files=csv_files,
        output_dir="/Volumes/Data/betterwhy_processed",
        remove_duplicates=True,
        max_workers=None  # None이면 자동으로 CPU 코어 수의 80% 사용
    )
    
    # 결과 분석
    if results:
        print(f"\n📊 최종 처리 통계:")
        print(f"   - 총 처리 시간: {format_duration(results['total_duration'])}")
        print(f"   - 파일당 평균: {format_duration(results['avg_duration_per_file'])}")
        print(f"   - 성공률: {(results['successful']/results['total_files']*100):.1f}%")
        print(f"   - 총 처리 행 수: {results['total_rows_processed']:,}행")
        print(f"   - 시간당 처리량: {results['total_rows_processed']/(results['total_duration']/3600):.0f}행/시간")
    
    # Ray 종료
    ray.shutdown()
    print("🔚 Ray 종료 완료")

2025-08-05 02:01:07,021	INFO worker.py:1821 -- Started a local Ray instance.


🚀 Ray 초기화 완료 (워커 수: 8)
📁 총 300개 파일 병렬 처리 시작
📂 출력 디렉토리: /Volumes/Data/betterwhy_processed
⏳ 모든 파일 병렬 처리 중...
(preproc_betterwhy_parallel pid=65066) 🔄 처리 시작: 1357rqwe_V000BL0011_IONIQ5 LONGRANGE_202207.csv
(ProgressTracker pid=65065) 📊 진행률: 0.3% | 성공: 0 | 실패: 1 | 예상 남은 시간: 5분 39.7초
(ProgressTracker pid=65065) 📊 진행률: 0.7% | 성공: 0 | 실패: 2 | 예상 남은 시간: 2분 58.5초
(ProgressTracker pid=65065) 📊 진행률: 1.0% | 성공: 0 | 실패: 3 | 예상 남은 시간: 2분 2.6초
(ProgressTracker pid=65065) 📊 진행률: 1.3% | 성공: 0 | 실패: 4 | 예상 남은 시간: 1분 41.3초
(ProgressTracker pid=65065) 📊 진행률: 1.7% | 성공: 0 | 실패: 5 | 예상 남은 시간: 1분 56.3초
(ProgressTracker pid=65065) 📊 진행률: 2.0% | 성공: 0 | 실패: 6 | 예상 남은 시간: 1분 39.8초
(ProgressTracker pid=65065) 📊 진행률: 2.3% | 성공: 0 | 실패: 7 | 예상 남은 시간: 1분 41.9초
(ProgressTracker pid=65065) 📊 진행률: 2.7% | 성공: 0 | 실패: 8 | 예상 남은 시간: 1분 41.5초
(preproc_betterwhy_parallel pid=65069) 📦 aim21c_V007AJ0000_NIRO LONGRANGE_201801.csv: 2개 배치 병렬 처리 완료, 결합 중...


(preprocess_batch_parallel pid=65070) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/532998891.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(preprocess_batch_parallel pid=65070) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/532998891.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(preprocess_batch_parallel pid=65070) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/532998891.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of 

(preproc_betterwhy_parallel pid=65070) 🔄 처리 시작: ajutaxi-9_V020BG0010_IONIQ 2019_201701.csv [repeated 14x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(preproc_betterwhy_parallel pid=65068) 📦 48625ff_V004CA0001_EV6 LONGRANGE_202210.csv: 3개 배치 병렬 처리 완료, 결합 중... [repeated 6x across cluster]


(preprocess_batch_parallel pid=65116) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/532998891.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()` [repeated 18x across cluster]


...

(preproc_betterwhy_parallel pid=67665) ✅ wntjdgml_V000CA0008_CASPER LONGRANGE_202408.csv: 완료 (19,822행, 8.1초)
(preproc_betterwhy_parallel pid=67883) ✅ whote564_V000CD0095_IONIQ5 LONGRANGE 2022_202311.csv: 완료 (45,543행, 29.6초)
(ProgressTracker pid=65065) 📊 진행률: 97.3% | 성공: 187 | 실패: 105 | 예상 남은 시간: 14.8초
(ProgressTracker pid=65065) 📊 진행률: 97.7% | 성공: 187 | 실패: 106 | 예상 남은 시간: 12.9초
(preproc_betterwhy_parallel pid=67665) 🔄 처리 시작: yousjun_V003BA0012_IONIQ5 LONGRANGE 2022_202302.csv
(preproc_betterwhy_parallel pid=67883) 🔄 처리 시작: zoh71z_V009BL0000_KONA LONGRANGE_201810.csv [repeated 4x across cluster]
(ProgressTracker pid=65065) 📊 진행률: 98.0% | 성공: 188 | 실패: 106 | 예상 남은 시간: 11.1초
(preproc_betterwhy_parallel pid=68077) ✅ wce4122_V000CB0008_EV6 LONGRANGE_202110.csv: 완료 (43,276행, 32.8초)
(ProgressTracker pid=65065) 📊 진행률: 98.3% | 성공: 189 | 실패: 106 | 예상 남은 시간: 9.2초
(ProgressTracker pid=65065) 📊 진행률: 98.7% | 성공: 190 | 실패: 106 | 예상 남은 시간: 7.4초
(preproc_betterwhy_parallel pid=67883) 📦 zoh71z_V009BL00

(preprocess_batch_parallel pid=68131) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/532998891.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()` [repeated 18x across cluster]


(preproc_betterwhy_parallel pid=67883) ✅ zoh71z_V009BL0000_KONA LONGRANGE_201810.csv: 완료 (50,678행, 11.3초) [repeated 3x across cluster]
(preproc_betterwhy_parallel pid=67770) 📦 yjtaxi-6_V003AL0003_IONIQ5 LONGRANGE 2022_202211.csv: 4개 배치 병렬 처리 완료, 결합 중...
(ProgressTracker pid=65065) 📊 진행률: 99.0% | 성공: 191 | 실패: 106 | 예상 남은 시간: 5.6초
(ProgressTracker pid=65065) 📊 진행률: 99.3% | 성공: 192 | 실패: 106 | 예상 남은 시간: 3.7초
(ProgressTracker pid=65065) 📊 진행률: 99.7% | 성공: 193 | 실패: 106 | 예상 남은 시간: 1.9초
(preproc_betterwhy_parallel pid=68212) ✅ xlos20_V004AK0000_EV6 LONGRANGE_202101.csv: 완료 (37,783행, 21.8초)
(preproc_betterwhy_parallel pid=67665) ✅ yousjun_V003BA0012_IONIQ5 LONGRANGE 2022_202302.csv: 완료 (57,775행, 19.4초)
(ProgressTracker pid=65065) 📊 진행률: 100.0% | 성공: 194 | 실패: 106 | 예상 남은 시간: 0.0초
(preproc_betterwhy_parallel pid=67770) ✅ yjtaxi-6_V003AL0003_IONIQ5 LONGRANGE 2022_202211.csv: 완료 (119,165행, 28.3초)

🎉 병렬 처리 완료!
📈 성공: 194개 | ❌ 실패: 106개
📊 총 처리 행 수: 8,626,476행
⏱️  총 소요시간: 9분 25.5초
⚡ 평균 파일당: 1.9초

❌

(preprocess_batch_parallel pid=68077) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/532998891.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()` [repeated 6x across cluster]


🔚 Ray 종료 완료


In [4]:
import glob
import os

# 검색할 기본 경로
base_path = '/Volumes/Data/betterwhy_origin'

# '2025'로 시작하는 폴더를 찾는 패턴
search_pattern = os.path.join(base_path, '2025*')

# glob.glob() 함수를 사용해 패턴에 맞는 모든 경로를 가져온 후,
# os.path.isdir() 함수로 폴더만 필터링하여 리스트에 담습니다.
folder_list = [f for f in glob.glob(search_pattern) if os.path.isdir(f)]

# 결과 확인
if folder_list:
    print(f"총 {len(folder_list)}개의 폴더를 찾았습니다.")
    print("찾은 폴더 목록:")
    for folder in folder_list:
        print(folder)
else:
    print("해당하는 폴더를 찾지 못했습니다.")

# 이제 'folder_list' 변수에 '2025'로 시작하는 폴더들의 경로가 리스트 형태로 담겨 있습니다.

총 36개의 폴더를 찾았습니다.
찾은 폴더 목록:
/Volumes/Data/betterwhy_origin/20250514_0520(추가_20대)
/Volumes/Data/betterwhy_origin/20250319_0325(200대)
/Volumes/Data/betterwhy_origin/20250611_0617(추가_20대)
/Volumes/Data/betterwhy_origin/20250514_0520(260대)
/Volumes/Data/betterwhy_origin/20250226_0304(200대)
/Volumes/Data/betterwhy_origin/20250430_0506(추가_20대)
/Volumes/Data/betterwhy_origin/20250702_0708(300대)
/Volumes/Data/betterwhy_origin/20250423_0429(240대)
/Volumes/Data/betterwhy_origin/20250604_0610(280대)
/Volumes/Data/betterwhy_origin/20250409_0415(추가_20대)
/Volumes/Data/betterwhy_origin/20250618_0624(300대)
/Volumes/Data/betterwhy_origin/20250326_0401(추가_20대)
/Volumes/Data/betterwhy_origin/20250409_0415(220대)
/Volumes/Data/betterwhy_origin/20250625_0701(300대)
/Volumes/Data/betterwhy_origin/20250108_0114(200대)
/Volumes/Data/betterwhy_origin/20250219_0225(200대_2대는_교체)
/Volumes/Data/betterwhy_origin/20250528_0603(280대)
/Volumes/Data/betterwhy_origin/20250416_0422(240대)
/Vol

In [ ]:
for folder in folder_list:
    csv_files = glob.glob(f"{folder}/**/*.csv", recursive=True)
    
    
    # 병렬 처리 실행 (CPU 코어 수의 80% 사용)
    results = process_multiple_files_parallel(
        csv_files=csv_files,
        output_dir="/Volumes/Data/betterwhy_processed",
        remove_duplicates=True,
        max_workers=None  # None이면 자동으로 CPU 코어 수의 80% 사용
    )

    # 결과 분석
    if results:
        print(f"\n📊 최종 처리 통계:")
        print(f"   - 총 처리 시간: {format_duration(results['total_duration'])}")
        print(f"   - 파일당 평균: {format_duration(results['avg_duration_per_file'])}")
        print(f"   - 성공률: {(results['successful']/results['total_files']*100):.1f}%")
        print(f"   - 총 처리 행 수: {results['total_rows_processed']:,}행")
        print(f"   - 시간당 처리량: {results['total_rows_processed']/(results['total_duration']/3600):.0f}행/시간")

    # Ray 종료
    ray.shutdown()
    print("🔚 Ray 종료 완료")

2025-08-05 02:15:24,296	INFO worker.py:1821 -- Started a local Ray instance.


🚀 Ray 초기화 완료 (워커 수: 8)
📁 총 20개 파일 병렬 처리 시작
📂 출력 디렉토리: /Volumes/Data/betterwhy_processed
⏳ 모든 파일 병렬 처리 중...
(preproc_betterwhy_parallel pid=70050) 🔄 처리 시작: smra1999_EV9_202401.csv
(preproc_betterwhy_parallel pid=70057) 🔄 처리 시작: day9672_IONIQ5 LONGRANGE 2022_202310.csv
(preproc_betterwhy_parallel pid=70048) 📦 9224_ST1_202411.csv: 1개 배치 병렬 처리 완료, 결합 중...
(preproc_betterwhy_parallel pid=70057) 📦 day9672_IONIQ5 LONGRANGE 2022_202310.csv: 2개 배치 병렬 처리 완료, 결합 중...
(preproc_betterwhy_parallel pid=70048) 🔄 처리 시작: 9224_ST1_202411.csv [repeated 6x across cluster]


(preprocess_batch_parallel pid=70098) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/1523317319.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(preprocess_batch_parallel pid=70098) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/1523317319.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(preprocess_batch_parallel pid=70098) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/1523317319.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result 

(preproc_betterwhy_parallel pid=70050) 📦 smra1999_EV9_202401.csv: 3개 배치 병렬 처리 완료, 결합 중... [repeated 6x across cluster]
(ProgressTracker pid=70049) 📊 진행률: 5.0% | 성공: 1 | 실패: 0 | 예상 남은 시간: 3분 55.7초
(preproc_betterwhy_parallel pid=70048) ✅ 9224_ST1_202411.csv: 완료 (7,794행, 10.7초)
(preproc_betterwhy_parallel pid=70048) 🔄 처리 시작: delpainus_IONIQ5 LONGRANGE 2022_202307.csv


(preprocess_batch_parallel pid=70097) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/1523317319.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()` [repeated 39x across cluster]


...

(preproc_betterwhy_parallel pid=88141) 📦 revu-n-34_GV70.csv: 2개 배치 병렬 처리 완료, 결합 중... [repeated 3x across cluster]
(preproc_betterwhy_parallel pid=88135) 파일 병합 중 오류 (Client V009BL0002): time data "2025-07-16 06:29:19.280" doesn't match format " %Y-%m-%d %H:%M:%S.%f", at position 24902. You might want to try:
(preproc_betterwhy_parallel pid=88135)     - passing `format` if your strings have a consistent format;
(preproc_betterwhy_parallel pid=88135)     - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
(preproc_betterwhy_parallel pid=88135)     - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
(preproc_betterwhy_parallel pid=88137) 📦 pgtaxi-15_IONIQ6 LONGRANGE.csv: 3개 배치 병렬 처리 완료, 결합 중... [repeated 2x across cluster]
(preproc_betterwhy_parallel pid=88135) 월별 저장 중 오류: Can only use .dt accessor with datetimelike values
(ProgressTracker pid=881

(preprocess_batch_parallel pid=88187) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/1523317319.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()` [repeated 21x across cluster]


(preproc_betterwhy_parallel pid=88140) 📦 dufdl1025_EV6 LONGRANGE_202404.csv: 3개 배치 병렬 처리 완료, 결합 중...
(preproc_betterwhy_parallel pid=88184) 🔄 처리 시작: ajutaxi-27_IONIQ 2019_201701.csv
(ProgressTracker pid=88139) 📊 진행률: 3.3% | 성공: 1 | 실패: 7 | 예상 남은 시간: 7분 23.1초
(preproc_betterwhy_parallel pid=88184) 🔄 처리 시작: ltgdg-14_BONGO3_2022.csv
(preproc_betterwhy_parallel pid=88138) 파일 병합 중 오류 (Client V011BE0024): time data "2025-07-16 02:04:47.180" doesn't match format " %Y-%m-%d %H:%M:%S.%f", at position 12006. You might want to try:
(preproc_betterwhy_parallel pid=88138)     - passing `format` if your strings have a consistent format;
(preproc_betterwhy_parallel pid=88138)     - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
(preproc_betterwhy_parallel pid=88138)     - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
(ProgressTracker pid=88139) 📊 진행률:

(preprocess_batch_parallel pid=88134) /var/folders/bv/f_x33dnx0_d87vg42cs2ff600000gn/T/ipykernel_63946/1523317319.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()` [repeated 24x across cluster]


(preproc_betterwhy_parallel pid=88184) 📦 ltgdg-14_BONGO3_2022.csv: 3개 배치 병렬 처리 완료, 결합 중...
(preproc_betterwhy_parallel pid=88134) 🔄 처리 시작: iamme77_IONIQ5 LONGRANGE 2022_202310.csv
(ProgressTracker pid=88139) 📊 진행률: 4.6% | 성공: 4 | 실패: 7 | 예상 남은 시간: 7분 1.1초
(preproc_betterwhy_parallel pid=88155) ✅ revu-n-68_EV6 LONGRANGE.csv: 완료 (18,800행, 17.6초)


In [ ]:
import os
import glob

# ⭐⭐ 1. 대상 폴더 경로를 설정하세요. ⭐⭐
# 현재 스크립트가 실행되는 폴더의 모든 CSV 파일을 대상으로 합니다.
# 만약 '/Volumes/Data/betterwhy_origin/' 폴더 내의 파일을 대상으로 한다면:
file_path_pattern = '/Volumes/Data/betterwhy_processed/**/*.csv'

# glob.glob() 함수를 사용하여 모든 CSV 파일 경로를 찾습니다.
# recursive=True를 사용하면 하위 폴더까지 재귀적으로 검색합니다.
csv_files = glob.glob(file_path_pattern, recursive=True)

# 변경된 파일 개수 카운트
renamed_count = 0

print(f"총 {len(csv_files)}개의 CSV 파일을 찾았습니다.")

# 2. 각 파일에 대해 반복문 실행
for old_file_path in csv_files:
    # 파일명에서 'clientid_' 접두사를 확인하고 제거합니다.
    # os.path.basename()은 경로에서 파일 이름만 추출합니다.
    old_filename = os.path.basename(old_file_path)

    # 파일 이름이 'clientid_'로 시작하는지 확인
    if old_filename.startswith('clientid_'):
        # 'clientid_'를 제거한 새로운 파일 이름 생성
        new_filename = old_filename.replace('clientid_', '', 1)

        # 새로운 파일의 전체 경로 생성
        # os.path.dirname()은 파일 경로에서 디렉토리 부분만 추출합니다.
        directory = os.path.dirname(old_file_path)
        new_file_path = os.path.join(directory, new_filename)

        # 파일 이름 변경 (rename)
        try:
            os.rename(old_file_path, new_file_path)
            print(f"이름 변경 성공: '{old_filename}' -> '{new_filename}'")
            renamed_count += 1
        except Exception as e:
            print(f"이름 변경 실패: '{old_filename}' -> '{new_filename}' - 오류: {e}")
            
print("\n--- 작업 완료 ---")
print(f"총 {renamed_count}개의 파일 이름을 변경했습니다.")